In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
sns.set()

seed_val = 2021
np.random.seed(seed_val)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
def check_representation(train_dataset, validation_dataset, test_dataset, label_name):
  # sort the data to check representation
  sorted_train = train_dataset.sort_index()
  sorted_test = test_dataset.sort_index()

  # sort the data to check representation
  sorted_train = train_dataset.sort_index()
  sorted_vali = validation_dataset.sort_index()
  sorted_test = test_dataset.sort_index()

  # check representation
  fig = plt.figure()
  fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(15,12))

  ax[0][0].scatter(x=np.arange(len(sorted_train)), y=sorted_train[label_name], marker='.', alpha=0.4)
  ax[0][0].set_title("Index Sorted Training Data")
  ax[0][1].scatter(x=np.arange(len(sorted_vali)), y=sorted_vali[label_name], marker='.', alpha=0.4)
  ax[0][1].set_title("Index Sorted Validation Data")
  ax[0][2].scatter(x=np.arange(len(sorted_test)), y=sorted_test[label_name], marker='.', alpha=0.4)
  ax[0][2].set_title("Index Sorted Testing Data")
  ax[0][0].set_ylabel(label_name)

  train1 = sorted_train.sort_values(label_name)
  test1 = sorted_test.sort_values(label_name)

  y_val = train1[int(len(train1)/2)-1:int(len(train1)/2)][label_name]
  ax[1][0].scatter(x=np.arange(len(train_dataset)), y=train1[label_name], marker='.', alpha=0.4)
  # vertical line
  ax[1][0].plot([len(train_dataset)/2, len(train_dataset)/2], [0,train_dataset[label_name].max()], '--', alpha=0.4, color="red")
  # horizontal line
  ax[1][0].plot([0, len(train_dataset)], [y_val,y_val], '--',  alpha=0.4, color="red")
  ax[1][0].set_title("Label Sorted Training Data")

  sorted_ds=sorted_vali.sort_values(label_name)
  y_val = sorted_ds[int(len(sorted_ds)/2)-1:int(len(sorted_ds)/2)][label_name]
  ax[1][1].scatter(x=np.arange(len(sorted_ds)), y=sorted_ds[label_name], marker='.', alpha=0.4)
  # vertical line
  ax[1][1].plot([len(sorted_ds)/2, len(sorted_ds)/2], [0,sorted_ds[label_name].max()], '--',  alpha=0.4, color="red")
  # horizontal line
  ax[1][1].plot([0, len(sorted_ds)], [y_val,y_val], '--',  alpha=0.4, color="red")
  ax[1][1].set_title("Label Sorted Train Data")

  sorted_ds=sorted_test.sort_values(label_name)
  y_val = sorted_ds[int(len(sorted_ds)/2)-1:int(len(sorted_ds)/2)][label_name]
  ax[1][2].scatter(x=np.arange(len(sorted_ds)), y=sorted_ds[label_name], marker='.', alpha=0.4)
  # vertical line
  ax[1][2].plot([len(sorted_ds)/2, len(sorted_ds)/2], [0,sorted_ds[label_name].max()], '--',  alpha=0.4, color="red")
  # horizontal line
  ax[1][2].plot([0, len(sorted_ds)], [y_val,y_val], '--',  alpha=0.4, color="red")
  ax[1][2].set_title("Label Sorted Test Data")
  ax[1][0].set_ylabel(label_name)

In [ ]:
train_dataset = pd.read_csv("Train.csv")
validation_dataset = pd.read_csv("Vali.csv")
test_dataset = pd.read_csv("Test.csv")
  
train_dataset.pop("Unnamed: 0")
validation_dataset.pop("Unnamed: 0")
test_dataset.pop("Unnamed: 0")

label_name = 'Nc'
in_features = 5
out_nodes = 1

check_representation(train_dataset, validation_dataset, test_dataset, label_name)

In [ ]:
sorted_train = train_dataset.sort_values(label_name)
sorted_vali = validation_dataset.sort_values(label_name)
sorted_test = test_dataset.sort_values(label_name)

# Generate the inputs and labels

# TRAIN DATA
train = sorted_train.copy()
train_features = train.to_numpy()[:,0:in_features]
train_labels = train.to_numpy()[:,in_features]
X_Sorted = train_features.copy()
Y_Sorted = train_labels.copy()
# train = sorted_train
train = sorted_train.sample(frac=1, random_state=seed_val)
train_features = train.to_numpy()[:,0:in_features]
train_labels = train.to_numpy()[:,in_features]
X = train_features.copy()
Y = train_labels.copy()


# TRAINING VALIDATION DATA
vali = sorted_vali
# vali = sorted_vali.sample(frac=1, random_state=seed_val)
vali_features = vali.to_numpy()[:,0:in_features]
vali_labels = vali.to_numpy()[:,in_features]
X_vali = vali_features.copy()
Y_vali = vali_labels.copy()

# TEST DATA
test = sorted_test
# test = sorted_test.sample(frac=1, random_state=seed_val)
test_features = test.sort_values(label_name).to_numpy()[:,0:in_features]
test_labels = test.sort_values(label_name).to_numpy()[:,in_features]
X_test = test_features.copy()
Y_test = test_labels.copy()

eval_set = [(X,Y), (X_vali, Y_vali)]
eval_set = [(X_test, Y_test), (X,Y), (X_vali, Y_vali)]

## MLP Class
This was easier to use at some stage, and the model was trained using it. However, it is not necessary to put the TF model in a class.

In [ ]:
class MLP_Regression():
    def __init__(self, input_shape, nodes=30, dropout=False, dropoutrate=0.2, seed=97, kernel_reg=None, use_bias=True, activation="relu", kernel_initializer='he_normal'):
        self.model = Sequential()
        self.model.add(Dense(nodes, input_shape=(input_shape,), activation=activation, use_bias=use_bias, kernel_regularizer=kernel_reg, kernel_initializer=kernel_initializer))
        if dropout:
            self.add_dropout(dropoutrate, seed)

    def learning_rate(self, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9):
        self.lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_lr, decay_steps=decay_steps, decay_rate=decay_rate)

    def SGDoptimizer(self, momentum=0.1, nesterov=False, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9):
        self.learning_rate(initial_lr, decay_steps, decay_rate)
        self.optimizer = keras.optimizers.SGD(learning_rate=self.lr_schedule, momentum=momentum, nesterov=nesterov)

    def ADAMoptimizer(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False):
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, amsgrad=amsgrad, name="Adam")
    
    def NADAMoptimizer(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07):
        self.optimizer = keras.optimizers.Nadam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, name="Nadam")

    def add_layer(self, num_nodes, kernel_reg=None, use_bias=True, activation="relu", kernel_initializer='he_normal'):
        self.model.add(Dense(num_nodes, activation=activation, use_bias=use_bias, kernel_regularizer=kernel_reg, kernel_initializer=kernel_initializer))
    
    def add_output_layer(self, out_nodes, kernel_reg=None, use_bias=True, activation="relu", kernel_initializer='he_normal'):
        self.model.add(Dense(out_nodes, activation=activation, use_bias=use_bias, kernel_regularizer=kernel_reg, kernel_initializer=kernel_initializer))

    def add_dropout(self, rate, seed):
        self.model.add(keras.layers.Dropout(rate=rate, seed=seed))

    def compileModel(self, optimizer, loss="mean_squared_error", metrics=["MSE","MAE", "MAPE"]):
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    def train_model(self, X, Y, early_stopping=True, patience=3, epochs=10, batch_size=500, vali_split=0.2, shuffle=False, loss_monitor="loss"):
        if early_stopping:
            earlystop_callback = EarlyStopping(monitor=loss_monitor, min_delta=0, patience=patience, mode="min", restore_best_weights=True)
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split, callbacks=[earlystop_callback], shuffle=shuffle)
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch
        else:
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split, shuffle=shuffle)
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch

    def show_training_errors(self, save_img=False, name=None):
        hist = self.history

        fig1 = plt.figure()
        fig1, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,12))

        ax[0][0].set_xlabel('Epoch')
        ax[0][0].set_ylabel('MSE')
        ax[0][0].plot(hist['epoch'], hist['MSE'], label='Train Error')
        ax[0][0].plot(hist['epoch'], hist['val_MSE'], label='Val Error')
        ax[0][0].legend()
        ax[0][0].set_title("MSE Error")

        ax[0][1].set_xlabel('Epoch')
        ax[0][1].set_ylabel('MAE')
        ax[0][1].plot(hist['epoch'], hist['MAE'], label='Train Error')
        ax[0][1].plot(hist['epoch'], hist['val_MAE'], label='Val Error')
        ax[0][1].legend()
        ax[0][1].set_title("MAE Error")

        ax[1][0].set_xlabel('Epoch')
        ax[1][0].set_ylabel('MAPE')
        ax[1][0].plot(hist['epoch'], hist['MAPE'], label='Train Mean Abs {} Error'.format("%"))
        ax[1][0].plot(hist['epoch'], hist['val_MAPE'], label='Val Mean Abs {} Error'.format("%"))
        ax[1][0].legend()
        ax[1][0].set_title("MAPE")
        if save_img:
          fig1.savefig("{}_E.png".format(name))


    def test(self, test_features, test_labels, save_img=False, name=None, label_name="Nc"):
        test_input = test_features
        test_output = np.reshape(test_labels, (-1,1))
        prediction = self.model.predict(test_input)
        error = test_output-prediction
        acc = (abs(error)/test_output)
        

        fig2 = plt.figure()
        fig2, ax = plt.subplots(nrows=3, figsize=(15,16))
        # ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
        # ax[0].plot(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4)
        ax[0].scatter(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4, marker='.')
        ax[0].scatter(np.arange(len(test_output)), test_output, label="TestData", marker='.')
        ax[0].legend()
        ax[0].set_title("Model Prediction")
        ax[0].set_ylabel(label_name)

        ax[1].scatter(np.arange(len(error)), error, label="test_label-pred", marker='.')
        ax[1].legend()
        ax[1].set_ylabel("Error")
        # ax[1].set_title("Model Prediction")

        ax[2].scatter(np.arange(len(acc)), acc*100, label="|(error/test_label)|*100", marker='.')
        ax[2].legend()
        ax[2].set_ylabel("PE")
        if save_img:
          fig2.savefig("{}_P.png".format(name))
    
    def retrainable_save(self, loc_and_name):
        self.model.save("{}".format(loc_and_name))    
        self.history.to_csv("{}.csv".format(loc_and_name))

    def results(self, test_features, test_labels, saveit=False, name=None):
        my_model.show_training_errors(save_img=saveit, name=name)
        my_model.test(test_features, test_labels, save_img=saveit, name=name)

In [ ]:
# reg = l1(0.02)
# reg = l1_l2(l1=0.02, l2=0.02)
# reg = l2(1e-5)
reg = None

# Create the Model
my_model = MLP_Regression(in_features, nodes=100, dropout=False, dropoutrate=0.1, kernel_reg=reg, use_bias=True, activation="relu")

# select optimizer
# my_model.SGDoptimizer(momentum=0.6, nesterov=True, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9)
# my_model.ADAMoptimizer(learning_rate=1e-3, beta_1=0.97)
my_model.NADAMoptimizer(learning_rate=8e-4, beta_1=0.99, beta_2=0.999)

# add layers
my_model.add_layer(50, kernel_reg=reg, use_bias=True, activation="sigmoid")
my_model.add_layer(25, kernel_reg=reg, use_bias=True, activation="sigmoid")
my_model.add_layer(13, kernel_reg=reg, use_bias=True, activation="relu")
# add output layer
my_model.add_output_layer(out_nodes, kernel_reg=reg, use_bias=True, activation="linear")

# compile and define loss metrics
my_model.compileModel(my_model.optimizer, loss="mean_absolute_percentage_error")

# The model
my_model.model.summary()

In [ ]:
#_______________________________________________________________________________________________________________________
# Train the model
if len(train_dataset) > len(dataset):
  print("Noisey Training")
else:
  print("Non-Noisey Training")
print()

print("Training has started on {} samples.".format(len(train_features)))

my_model.train_model(X,Y, epochs=200, patience=15, batch_size=8, shuffle=True, early_stopping=True, loss_monitor="val_loss", vali_split=0.1)


## Get Results
Run the 2nd cell after this to save the model. The results save 'X_E' for the erros and 'X_P' for the predictions where X is defined by the user. Can change this in the class as is necessary.

In [ ]:
_NAME = "NNModel4"

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
my_model.retrainable_save(loc_and_name=_NAME)

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
_NAME1 = _NAME+"Train"

my_model.results(X_Sorted, Y_Sorted, saveit=True, name=_NAME1)

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
_NAME2 = _NAME+"Vali"

my_model.results(X_vali, Y_vali, saveit=True, name=_NAME2)

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
_NAME3 = _NAME+"Test"

my_model.results(X_test, Y_test, saveit=True, name=_NAME3)

In [ ]:
my_model.model.summary()

## Second Round of Training?
This can be used the same as the first, but using the weights from the previous model as a starting point. Not recommended.

# Loaded Model Predictor

In [ ]:
class LoadedMLP_Regression():
    def __init__(self, model):
        self.model = model

    def learning_rate(self, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9):
        self.lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_lr, decay_steps=decay_steps, decay_rate=decay_rate)

    def SGDoptimizer(self, momentum=0.1, nesterov=False, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9):
        self.learning_rate(initial_lr, decay_steps, decay_rate)
        self.optimizer = keras.optimizers.SGD(learning_rate=self.lr_schedule, momentum=momentum, nesterov=nesterov)

    def ADAMoptimizer(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False):
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, amsgrad=amsgrad, name="Adam")
    
    def NADAMoptimizer(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07):
        self.optimizer = keras.optimizers.Nadam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, name="Nadam")

    def add_layer(self, num_nodes, kernel_reg=None, use_bias=True, activation="relu", kernel_initializer='he_normal'):
        self.model.add(Dense(num_nodes, activation=activation, use_bias=use_bias, kernel_regularizer=kernel_reg, kernel_initializer=kernel_initializer))
    
    def add_output_layer(self, out_nodes, kernel_reg=None, use_bias=True, activation="relu", kernel_initializer='he_normal'):
        self.model.add(Dense(out_nodes, activation=activation, use_bias=use_bias, kernel_regularizer=kernel_reg, kernel_initializer=kernel_initializer))

    def add_dropout(self, rate, seed):
        self.model.add(keras.layers.Dropout(rate=rate, seed=seed))

    def compileModel(self, optimizer, loss="mean_squared_error", metrics=["MSE","MAE", "MAPE"]):
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    def train_model(self, X, Y, early_stopping=True, patience=3, epochs=10, batch_size=500, vali_split=0.2, shuffle=False, loss_monitor="loss"):
        if early_stopping:
            earlystop_callback = EarlyStopping(monitor=loss_monitor, min_delta=0, patience=patience, mode="min", restore_best_weights=True)
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split, callbacks=[earlystop_callback], shuffle=shuffle)
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch
        else:
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split, shuffle=shuffle)
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch

    def show_training_errors(self, save_img=False, name=None):
        hist = self.history

        fig1 = plt.figure()
        fig1, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,12))

        ax[0][0].set_xlabel('Epoch')
        ax[0][0].set_ylabel('MSE')
        ax[0][0].plot(hist['epoch'], hist['MSE'], label='Train Error')
        ax[0][0].plot(hist['epoch'], hist['val_MSE'], label='Val Error')
        ax[0][0].legend()
        ax[0][0].set_title("MSE Error")

        ax[0][1].set_xlabel('Epoch')
        ax[0][1].set_ylabel('MAE')
        ax[0][1].plot(hist['epoch'], hist['MAE'], label='Train Error')
        ax[0][1].plot(hist['epoch'], hist['val_MAE'], label='Val Error')
        ax[0][1].legend()
        ax[0][1].set_title("MAE Error")

        ax[1][0].set_xlabel('Epoch')
        ax[1][0].set_ylabel('MAPE')
        ax[1][0].plot(hist['epoch'], hist['MAPE'], label='Train Mean Abs {} Error'.format("%"))
        ax[1][0].plot(hist['epoch'], hist['val_MAPE'], label='Val Mean Abs {} Error'.format("%"))
        ax[1][0].legend()
        ax[1][0].set_title("MAPE")
        if save_img:
          fig1.savefig("{}_E.png".format(name))


    def test(self, test_features, test_labels, save_img=False, name=None, label_name="Nc"):
        test_input = test_features
        test_output = np.reshape(test_labels, (-1,1))
        prediction = self.model.predict(test_input)
        error = test_output-prediction
        acc = (abs(error)/test_output)
        

        fig2 = plt.figure()
        fig2, ax = plt.subplots(nrows=3, figsize=(15,16))
        # ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
        # ax[0].plot(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4)
        ax[0].scatter(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4, marker='.')
        ax[0].scatter(np.arange(len(test_output)), test_output, label="TestData", marker='.')
        ax[0].legend()
        ax[0].set_title("Model Prediction")
        ax[0].set_ylabel(label_name)

        ax[1].scatter(np.arange(len(error)), error, label="test_label-pred", marker='.')
        ax[1].legend()
        ax[1].set_ylabel("Error")
        # ax[1].set_title("Model Prediction")

        ax[2].scatter(np.arange(len(acc)), acc*100, label="|(error/test_label)|*100", marker='.')
        ax[2].legend()
        ax[2].set_ylabel("PE")
        if save_img:
          fig2.savefig("{}_P.png".format(name))
    
    def retrainable_save(self, loc_and_name):
        self.model.save("{}".format(loc_and_name))    
        self.history.to_csv("{}.csv".format(loc_and_name))

    def results(self, test_features, test_labels, saveit=False, name=None):
        self.show_training_errors(save_img=saveit, name=name)
        self.test(test_features, test_labels, save_img=saveit, name=name)

In [ ]:
train_dataset_old = train_dataset.copy()

In [ ]:
load = keras.models.load_model("/"+_NAME)
# load model
loaded_model = LoadedMLP_Regression(load)

# choose optimizer
# loaded_model.SGDoptimizer(momentum=0.3, nesterov=True, initial_lr=1e-7, decay_steps=1e5, decay_rate=0.95)
# loaded_model.ADAMoptimizer(learning_rate=1e-5, beta_1=0.97)
loaded_model.NADAMoptimizer(learning_rate=1e-6, beta_1=0.99)

# compile and define loss metrics
loaded_model.compileModel(loaded_model.optimizer, loss="mean_absolute_percentage_error")

In [ ]:
loaded_model.model.summary()

In [ ]:

#_______________________________________________________________________________________________________________________
# Train the model
if len(train_dataset) > len(dataset):
  print("Noisey Training")
else:
  print("Non-Noisey Training")
print()

print("Re-Training has started on {} samples.".format(len(train_features)))

loaded_model.train_model(X,Y, epochs=400, patience=20, batch_size=8, shuffle=True, early_stopping=True, loss_monitor="val_loss", vali_split=0.2)

## Get Results
Basically the same as before, except with added suffixes.

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
_NAME_2 = _NAME+"Retrain"

In [ ]:
#_______________________________________________________________________________________________________________________
# Save the model
my_model.retrainable_save(loc_and_name=_NAME_2)

In [ ]:
my_model.retrainable_save(loc_and_name=_NAME_2)

In [ ]:
_NAME3 = _NAME_2+"Tr"

my_model.results(X.copy(), Y.copy(), saveit=True, name=_NAME3)

In [ ]:
#_______________________________________________________________________________________________________________________
# Get the results on the test or training data
# Use TEST DATA
_NAME3 = _NAME_2+"V"

my_model.results(X_vali, Y_vali, saveit=True, name=_NAME3)

In [ ]:
#_______________________________________________________________________________________________________________________
# Get the results on the test or training data
# Use TEST DATA
_NAME3 = _NAME_2+"Te"

my_model.results(X_test.copy(), Y_test.copy(), saveit=True, name=_NAME3)